In [1]:
import numpy as np
import pandas as pd
import os
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [4]:
def get_dir_list(dir_path):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite, 
    creates list of path names"""
    
    path_names = []
    dirs = os.listdir(dir_path)
    for file in dirs: 
        path_names.append(dir_path + "/" + file)

    return path_names   

In [7]:
def get_file_info(file_path_name, session_name):
    """takes in a file path and session name (should also be contained in file name),
    finds the animal_number and session and saves each accordingly. 
    each file should be named with animal number and exp (e.g. 100_CPA.Table.1.selections)"""
    
    path_split = file_path_name.split('/')
    file_split = path_split[-1].split('.')
    animal_number, session = file_split[0].split('_')

    return animal_number, session

In [8]:
def create_animal_df(file_path_name, animal_number, session):
    """takes in a file path, animal number and session and creates data frame.
    adds animal number, session, and time stamp columns"""
    
    data = pd.read_table(file_path_name)
    data = pd.DataFrame(data = data)
    
    data['animal_number'] = [animal_number]*data.shape[0]
    data['session'] = [session]*data.shape[0]
    
    #Determine closest time stamp of each annotation and add as column to df
    data['Begin Time (s)_1000'] = data['Begin Time (s)']*1000
    time_lambda = lambda a: (a // 22.5)*22.5
    data['time_stamp'] = time_lambda(data['Begin Time (s)_1000'])
    
    data = data[['animal_number', 'session', 'time_stamp', 'Annotation']]

    return data

In [9]:
def create_annot_df(dir_path, session_name):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite,
    uses get_dir_list, get_file_info, and create_animal_df functions 
    to create a single data frame containing annotations for each file in dir_path"""
    
    annot_df = pd.DataFrame()
    
    path_names = get_dir_list(dir_path)
    
    for path in path_names:
        animal_number, session = get_file_info(path, session_name)
        animal_df = create_animal_df(path, animal_number, session)
        annot_df = annot_df.append(animal_df)
    
    #save as csv using name of dir_path
    annot_df.to_csv(str('annot_df_' + str(dir_path.split('/')[-1]) + '.csv'))
    
    return annot_df

In [10]:
dir_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/Annotation_tables/3x_CPA_pair_tables"

In [11]:
annot_df = create_annot_df(dir_path, 'CPApair')
print(annot_df.shape)

(459, 4)
